In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import glob

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.utils import to_categorical

import cv2

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import random

import tensorflow as tf

from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import InceptionV3

from tensorflow.keras import backend as K

from sklearn.model_selection import StratifiedKFold

import math

In [ ]:
from scipy import ndimage

"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""

tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

In [ ]:
from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import Recall

from tensorflow.keras.metrics import TruePositives
from tensorflow.keras.metrics import FalsePositives

from tensorflow.keras.metrics import TrueNegatives
from tensorflow.keras.metrics import FalseNegatives

In [ ]:
def get_X_y(file_path_list, dim):
    image_list = []
    teeth_class = []
    for path in file_path_list:
        img = cv2.imread(path)
        img = cv2.resize(img, (dim,dim)) / 255
        image_list.append(img)
        if path.split("/")[-3] == "Modern Teeth":
            teeth_class.append(0)
        else:
            teeth_class.append(1)
    
    X = np.array(image_list)
    y = np.array(teeth_class)
    print(np.unique(y, return_counts = True))
    y = to_categorical(y, 2)
    return X, y


def gmean(y_true, y_pred):
    pos_score = (pos_recall(y_true, y_pred))
    neg_score = (neg_recall(y_true, y_pred))
    print(pos_score)
    print(neg_score)
    return K.sqrt(K.prod([pos_score, neg_score]))

def rotateImages(X, Y):
    """
    Takes in original image and label
    Returns array of 7 rotated arrays + labels
    """ 
    
    # Store rotated images and labels for appending to training
    x_rotated = [] # lists are faster, convert after
    y_rotated = []
    
    DEGREE = 45
    
    # Handle Singular input
    multiple = True if len(X.shape) > 3 else False
    
    for idx, img in enumerate(X):
        try:
            y = Y[idx]
        except:
            y = Y

        # Seven image rotations
        for turn in range(1, 8):
            img = img if multiple else X
            
            rotation = turn * DEGREE
            r_img = ndimage.rotate(img, rotation, reshape=False)
            
            x_rotated.append(r_img)
            y_rotated.append(y)
            
        if not multiple: break
            
    return np.array(x_rotated), np.array(y_rotated)

def flipImages(X, Y):
    """
    Takes in original image and label
    np.flip also flips the array sequence itself
    So we have to flip the y-value array
    Returns array of 2 flipped arrays + labels
    """
    
    # Flip vertically, along with labels
    vert_flip = np.flip(X)
    vert_flipped_labels = np.flip(Y)

    # Apply to new dataset
    new_x_train = vert_flip
    new_y_train = vert_flipped_labels

    # Flip horizontally (no need to flip labels)
    horz_flip = np.fliplr(X)
    
    # Apply to dataset and return
    new_x_train = np.concatenate((new_x_train, horz_flip))
    new_y_train = np.concatenate((new_y_train, Y))
    
    return new_x_train, new_y_train

def give_simple_model(input_shape, drop_rate):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3,3), activation = 'relu', input_shape = input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(drop_rate, seed = 42))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(16, (3,3), activation = 'relu',))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(drop_rate, seed = 42))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(16, (3,3), activation = 'relu',))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(drop_rate, seed = 42))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(16, (2,2), activation = 'relu',))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(drop_rate, seed = 42))
    model.add(layers.MaxPooling2D((2,2)))


    model.add(layers.Flatten())
    model.add(layers.Dense(4, activation = 'relu',))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(2, activation = 'softmax',))

    #print(model.summary())
    return model

def return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, class_weight = {0: 0.5, 1: 0.5}, augment = False, custom_model = False, verbosity = 0):
    # Neural Network Parameters
    input_shape = (dim, dim, 3)
    adam = optimizers.Adam(learning_rate=lr_rate)
    input_shape = (dim, dim, 3)
    historical_vals = []
    count = 1
    for train_index, test_index in skf.split(X_place_holder, label):
        print(count)
        train_path_list = []
        test_path_list = []

        train_paths = list_of_images[train_index]
        test_paths = list_of_images[test_index]
        if custom_model == False:
            model = give_simple_model(input_shape = input_shape, drop_rate = drop_rate)
        elif custom_model == "VGG16":
            base_vgg = VGG16(weights='imagenet', include_top = False, input_shape=input_shape)
            last = base_vgg.layers[-2].output
            out = GlobalAveragePooling2D()(last)
            out = layers.Dense(128, activation='relu')(out)
            out = layers.Dense(64, activation='relu')(out)
            out = layers.Dense(8, activation='relu')(out)
            #model.add(layers.Dropout(drop_rate))
            total_classes = 2
            predictions = layers.Dense(total_classes, activation='softmax')(out)

            # Stack the two models (VGG16 and custom layers) on top of each other
            model = Model(inputs=base_vgg.input, outputs=predictions)

            # Freeze all our base_inception layers and train the last ones
            for layer in base_vgg.layers:
                layer.trainable = False
        elif custom_model == "VGG19":
            base_vgg = VGG19(weights='imagenet', include_top = False, input_shape=input_shape)
            last = base_vgg.layers[-2].output
            out = GlobalAveragePooling2D()(last)
            out = layers.Dense(128, activation='relu')(out)
            out = layers.Dense(64, activation='relu')(out)
            out = layers.Dense(8, activation='relu')(out)
            total_classes = 2
            predictions = layers.Dense(total_classes, activation='softmax')(out)

            # Stack the two models (VGG16 and custom layers) on top of each other
            model = Model(inputs=base_vgg.input, outputs=predictions)

            # Freeze all our base_inception layers and train the last ones
            for layer in base_vgg.layers:
                layer.trainable = False
                
        elif custom_model == "ResNet50":
            base_vgg = ResNet50(weights='imagenet', include_top = False, input_shape=input_shape)
            last = base_vgg.layers[-2].output
            out = GlobalAveragePooling2D()(last)
            out = layers.Dense(128, activation='relu')(out)
            out = layers.Dense(64, activation='relu')(out)
            out = layers.Dense(8, activation='relu')(out)
            total_classes = 2
            predictions = layers.Dense(total_classes, activation='softmax')(out)

            # Stack the two models (VGG16 and custom layers) on top of each other
            model = Model(inputs=base_vgg.input, outputs=predictions)

            # Freeze all our base_inception layers and train the last ones
            for layer in base_vgg.layers:
                layer.trainable = False
             
        elif custom_model == "Xception":
            base_vgg = Xception(weights='imagenet', include_top = False, input_shape=input_shape)
            last = base_vgg.layers[-2].output
            out = GlobalAveragePooling2D()(last)
            out = layers.Dense(128, activation='relu')(out)
            out = layers.Dense(64, activation='relu')(out)
            out = layers.Dense(8, activation='relu')(out)
            total_classes = 2
            predictions = layers.Dense(total_classes, activation='softmax')(out)

            # Stack the two models (VGG16 and custom layers) on top of each other
            model = Model(inputs=base_vgg.input, outputs=predictions)

            # Freeze all our base_inception layers and train the last ones
            for layer in base_vgg.layers:
                layer.trainable = False
                
        elif custom_model == "InceptionV3":
            base_vgg = InceptionV3(weights='imagenet', include_top = False, input_shape=input_shape)
            last = base_vgg.layers[-2].output
            out = GlobalAveragePooling2D()(last)
            out = layers.Dense(128, activation='relu')(out)
            out = layers.Dense(64, activation='relu')(out)
            out = layers.Dense(8, activation='relu')(out)
            total_classes = 2
            predictions = layers.Dense(total_classes, activation='softmax')(out)

            # Stack the two models (VGG16 and custom layers) on top of each other
            model = Model(inputs=base_vgg.input, outputs=predictions)

            # Freeze all our base_inception layers and train the last ones
            for layer in base_vgg.layers:
                layer.trainable = False
                
        for train_path in train_paths:
            train_path_list.extend(glob.glob(train_path + "/*"))    
        for test_path in test_paths:
            test_path_list.extend(glob.glob(test_path + "/*"))  

        X_train, y_train = get_X_y(train_path_list, dim)
        if augment == True:
            x_flipped, y_flipped = flipImages(X_train, y_train)
            x_rotated, y_rotated = rotateImages(X_train, y_train)
            # Concat all arrays together to make one big happy array
            X_train = np.concatenate((X_train, x_flipped, x_rotated))
            y_train = np.concatenate((y_train, y_flipped, y_rotated))
        
        X_test, y_test = get_X_y(test_path_list, dim)

        model.compile(optimizer=adam, 
                      loss='binary_crossentropy',
                      metrics=['accuracy', pos_recall, neg_recall, pos_precision, neg_precision]
                     )
        result_gen = model.fit(X_train, y_train, batch_size = 4, epochs = epochs, verbose = verbosity, validation_data=(X_test, y_test),class_weight = class_weight)
        curr_results = result_gen.history
        historical_vals.append(list(curr_results.values()))
        count = count + 1
    result = np.mean(historical_vals, axis = 0)
    result = pd.DataFrame(result).T
    result.columns = list(result_gen.history.keys())
    result['Val Geometric Mean'] = ((result['val_pos_recall']) * (result['val_neg_recall'])).apply(np.sqrt)
    left = ((result['val_pos_recall']) * (result['val_neg_recall']) * (result['val_neg_precision'])* (result['val_pos_precision'])).apply(np.sqrt)
    right = ((1 - (result['val_pos_recall'])) * (1 - (result['val_neg_recall'])) * (1 - (result['val_neg_precision'])) * (1 - (result['val_pos_precision']))).apply(np.sqrt)
    result['Val MCC'] = left - right
        
    result = result[['val_accuracy', 'val_pos_recall','val_neg_recall', 'val_pos_precision' ,'val_neg_precision','Val Geometric Mean', 'Val MCC']]
    return result.sort_values('Val Geometric Mean', ascending = False).round(4) 

In [ ]:


pos_precision = Precision(thresholds=None, top_k=None, class_id=1, name='pos_precision', dtype=None)
neg_precision = Precision(thresholds=None, top_k=None, class_id=0, name='neg_precision', dtype=None)

pos_recall = Recall(thresholds=None, top_k=None, class_id=1, name='pos_recall', dtype=None)
neg_recall = Recall(thresholds=None, top_k=None, class_id=0, name='neg_recall', dtype=None)

image_path = '/kaggle/input/forensic-images/Forensic Data/Ancient Teeth/Lot 825.2/Lot 825.2 Dentin 3.tif'
path = "/kaggle/input/forensic-images/Forensic Data/"

#Test read and show
img = cv2.imread(image_path,0)

dim = 256

#Gets a list of unique images
list_of_images = []
image_list = []
teeth_class = []
missed = 0
for dirpath, dirs, files in os.walk(path):
    for filename in files:
        list_of_images.append(dirpath)
        
list_of_images = np.array(list(set(list_of_images)))
label = [image.split("/")[-2] for image in list_of_images]
label = pd.Series(label).replace(["Modern Teeth", "Ancient Teeth"], [0, 1]).values

print(np.unique(label, return_counts = True))

X_place_holder = np.ones(len(label))

print(label)

neg = 25
pos = 4
total = 29

skf = StratifiedKFold(n_splits=4)

# Imbalance Data Reweighting

In [ ]:
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

In [ ]:
drop_rate = 0.30
lr_rate = 0.01
epochs = 200

# Transfer Learning [Base Case Custom]

In [ ]:
base_case = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images)

#weighted base case
simple_weights = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, class_weight)

#Augmented Data No Weights
aug_data_no_weights = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, augment = True)

#Augmented Data with Weights
aug_simple_weights = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, class_weight, augment = True)

In [ ]:
base_case.to_csv("Base_Custom_NoWeights.csv",index=True)
simple_weights.to_csv("Base_Custom_Weights.csv",index=True)
aug_data_no_weights.to_csv("Aug_Custom_NoWeights.csv",index=True)
aug_simple_weights.to_csv("Aug_Custom_Weights.csv",index=True)

# Transfer Learning [VGG16]

In [ ]:
#Base Case
base_case = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, custom_model = "VGG16")

#Weighted base case
simple_weights = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, class_weight, custom_model = 'VGG16')

#Augmented Data No Weights
aug_data_no_weights = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, augment = True, custom_model = 'VGG16')

#Data with Weights
trans_aug_simple_weights = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, class_weight, augment = True, custom_model = 'VGG16')

In [ ]:
base_case.to_csv("VGG16_NoWeights.csv",index=True)
simple_weights.to_csv("VGG16_Weights.csv",index=True)
aug_data_no_weights.to_csv("Aug_VGG16_NoWeights.csv",index=True)
aug_simple_weights.to_csv("Aug_VGG16_Weights.csv",index=True)

# Transfer Learning [VGG19]

In [ ]:
#Base Case
base_case = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, custom_model = "VGG19", verbosity = 0)

#Weighted base case
simple_weights = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, class_weight, custom_model = 'VGG19')

#Augmented Data No Weights
aug_data_no_weights = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, augment = True, custom_model = 'VGG19')

#Data with Weights
trans_aug_simple_weights = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, class_weight, augment = True, custom_model = 'VGG19')

In [ ]:
base_case.to_csv("VGG19_NoWeights.csv",index=True)
simple_weights.to_csv("VGG19_Weights.csv",index=True)
aug_data_no_weights.to_csv("Aug_VGG19_NoWeights.csv",index=True)
aug_simple_weights.to_csv("Aug_VGG19_Weights.csv",index=True)

# Transfer Learning [ResNet50]

In [ ]:
#Base Case
base_case = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, custom_model = "ResNet50", verbosity = 0)

#Weighted base case
simple_weights = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, class_weight, custom_model = 'ResNet50')

#Augmented Data No Weights
aug_data_no_weights = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, augment = True, custom_model = 'ResNet50')

#Data with Weights
trans_aug_simple_weights = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, class_weight, augment = True, custom_model = 'ResNet50')

In [ ]:
base_case.to_csv("ResNet50_NoWeights.csv",index=True)
simple_weights.to_csv("ResNet50_Weights.csv",index=True)
aug_data_no_weights.to_csv("Aug_ResNet50_NoWeights.csv",index=True)
aug_simple_weights.to_csv("Aug_ResNet50_Weights.csv",index=True)

# Transfer Learning [Xception]

In [ ]:
#Base Case
base_case = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, custom_model = "Xception", verbosity = 0)

#Weighted base case
simple_weights = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, class_weight, custom_model = 'Xception', verbosity = 0)

#Augmented Data No Weights
aug_data_no_weights = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, augment = True, custom_model = 'Xception', verbosity = 0)

#Data with Weights
trans_aug_simple_weights = return_results(drop_rate, dim, lr_rate, epochs, X_place_holder, label, list_of_images, class_weight, augment = True, custom_model = 'Xception', verbosity = 0)

In [ ]:
base_case.to_csv("Xception_NoWeights.csv",index=True)
simple_weights.to_csv("Xception_Weights.csv",index=True)
aug_data_no_weights.to_csv("Aug_Xception_NoWeights.csv",index=True)
aug_simple_weights.to_csv("Aug_Xception_Weights.csv",index=True)